In [70]:
import copy, numpy as np

data = open('HP1.txt','r', encoding="utf8").read();
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size,", ",vocab_size)

431673 ,  79


In [71]:
# Dictionary of input chars & indices
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'-': 0, 'n': 1, 'Q': 2, '9': 3, 'M': 4, 'm': 5, 'k': 6, '?': 7, 'a': 8, '\n': 9, 'R': 10, 't': 11, 'Y': 12, '(': 13, '!': 14, ')': 15, 'I': 16, '*': 17, '1': 18, '\\': 19, '6': 20, 'o': 21, 'E': 22, 'S': 23, '4': 24, 'c': 25, 'x': 26, 'q': 27, ' ': 28, 'b': 29, 'h': 30, 'U': 31, 'T': 32, 'K': 33, 'v': 34, 'N': 35, 'G': 36, 'X': 37, 'y': 38, 'g': 39, "'": 40, '7': 41, 'A': 42, 'Z': 43, 'W': 44, 'w': 45, 'd': 46, 'p': 47, 'F': 48, 'e': 49, '~': 50, 's': 51, '8': 52, 'D': 53, 'L': 54, ';': 55, 'V': 56, 'C': 57, 'j': 58, '"': 59, 'i': 60, '\t': 61, 'z': 62, 'O': 63, '3': 64, '0': 65, 'r': 66, 'P': 67, 'u': 68, ':': 69, 'f': 70, '5': 71, '.': 72, 'H': 73, 'B': 74, ',': 75, '2': 76, 'J': 77, 'l': 78}
{0: '-', 1: 'n', 2: 'Q', 3: '9', 4: 'M', 5: 'm', 6: 'k', 7: '?', 8: 'a', 9: '\n', 10: 'R', 11: 't', 12: 'Y', 13: '(', 14: '!', 15: ')', 16: 'I', 17: '*', 18: '1', 19: '\\', 20: '6', 21: 'o', 22: 'E', 23: 'S', 24: '4', 25: 'c', 26: 'x', 27: 'q', 28: ' ', 29: 'b', 30: 'h', 31: 'U', 32: 'T', 33: '

In [72]:
# demo of onehot encoding
vector_for_char_a = np.zeros((vocab_size,1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [73]:
# Hyperparams
hidden_size = 100
hidden_size2 =50
seq_length = 25
learning_rate = 1e-1
# the lower the learning rate, the quicker the network abandons old belief for new input
# e.g. train images on dogs, give a cat, low learning rate will consider cat is anormally rather than dog

# Model params
Wxh1 = np.random.randn(hidden_size,  vocab_size)* 0.01 # input to hidden (input is onehot encoded)
Wh1h2 = np.random.randn(hidden_size2, hidden_size)* 0.01 # hidden 1 to hidden 2
Wh2h3 = np.random.randn(hidden_size, hidden_size2)* 0.01 # hidden 2 to hidden 3
Wh3y = np.random.randn(vocab_size,  hidden_size)* 0.01 # hidden to output(decode the output)

Wh1h1 = np.random.randn(hidden_size, hidden_size)* 0.01 # recurrent hidden .
Wh2h2 = np.random.randn(hidden_size2, hidden_size2)* 0.01 # recurrent hidden .
Wh3h3 = np.random.randn(hidden_size, hidden_size)* 0.01 # recurrent hidden .

Bxh1 = np.zeros((hidden_size,1))
Bh1h2 = np.zeros((hidden_size2,1))
Bh2h3 = np.zeros((hidden_size,1))
Bh3y = np.zeros((vocab_size,1))


In [38]:
# softmax helper
def softmax(seq):
    return np.exp(seq)/ np.sum(np.exp(seq))

def softmax_array(two_D_seq,t):
    return np.exp(two_D_seq[t])/ np.sum(np.exp(two_D_seq[t]))

In [65]:
# Loss function - training
def lossFunction(inputs, targets, prev_hidden1, prev_hidden2, prev_hidden3):
    # p is softmax probability
    xs, h1s, h2s, h3s, ys, ps = {},{},{},{},{},{};
    
    h1s[-1] = copy.deepcopy(prev_hidden1)
    h2s[-1] = copy.deepcopy(prev_hidden2)
    h3s[-1] = copy.deepcopy(prev_hidden3)
    loss = 0;
    
    # Fwd pass    
    for t in range(len(inputs)):
        # One hot encoding for the input char using our dictionary
        xs[t] = np.zeros((vocab_size,1));
        xs[t][inputs[t]] = 1; 
        
        h1s[t] = np.tanh(np.dot(Wxh1,xs[t]) + np.dot(Wh1h1,h1s[t-1]) + Bxh1);
        h2s[t] = np.tanh(np.dot(Wh1h2,h1s[t]) + np.dot(Wh2h2,h2s[t-1]) + Bh1h2);
        h3s[t] = np.tanh(np.dot(Wh2h3,h2s[t]) + np.dot(Wh3h3,h3s[t-1]) + Bh2h3);
        
        ys[t] = np.dot(Wh3y,h3s[t]) + Bh3y;
        ps[t] = softmax(ys[t]);
        char_idx = targets[t]
        loss += -np.log(ps[t][char_idx,0]) 
        # ps[t][targets[t]] is the prob. node corrs. to. t_th char in the label array

        
    # Gradient value holders
    dWxh1, dWh1h2, dWh2h3, dWh3y = np.zeros_like(Wxh1 ),np.zeros_like(Wh1h2),np.zeros_like(Wh2h3),np.zeros_like(Wh2y)
    dWh1h1,dWh2h2, dWh3h3        = np.zeros_like(Wh1h1),np.zeros_like(Wh2h2),np.zeros_like(Wh3h3)
    dBxh1, dBh1h2, dBh2h3, dBh3y = np.zeros_like(Bxh1 ),np.zeros_like(Bh1h2),np.zeros_like(Bh2h3),np.zeros_like(Bh3y)
    dh1next = np.zeros_like(h1s[0])
    dh2next = np.zeros_like(h2s[0])
    dh3next = np.zeros_like(h3s[0])
    
    # Bwd pass
    for t in reversed(range(len(inputs))):
        
        # prob. p to output y
        dy = copy.deepcopy(ps[t])
        dy[targets[t]] -= 1 # this is how we calculate loss using onehot encoding
        
        # y to h3 
        dWh3y += np.dot(dy, h3s[t].T);        
        dBh3y += dy # derivative w.r.t bias is 1        
        dh3 = np.dot(Wh3y.T,dy) + dh3next # back prop the error from y into h3
        
        # h3 to h2 and h3_prev
        dh3raw = (1-h3s[t]*h3s[t])*dh3 # back prop thru tanh
        dBh2h3 += dh3raw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
        dWh2h3 += np.dot(dh3raw, h2s[t].T) # derivative of Wx+b w.r.t W is x
        dWh3h3 += np.dot(dh3raw,h3s[t-1].T)
        dh3next = np.dot(Wh3h3.T, dh3raw)
        
        dh2 = np.dot(Wh2h3.T,dh3) + dh2next # back prop the error from h3 into h2     
        
        # h2 to h1 and h2_prev
        dh2raw = (1-h2s[t]*h2s[t])*dh2 # back prop thru tanh
        dBh1h2 += dh2raw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
        dWh1h2 += np.dot(dh2raw, h1s[t].T) # derivative of Wx+b w.r.t W is x
        dWh2h2 += np.dot(dh2raw,h2s[t-1].T)
        dh2next = np.dot(Wh2h2.T, dh2raw)
        
        dh1 = np.dot(Wh1h2.T,dh2) + dh1next # back prop the error from y into h2
        
        # h1 to x and h1_prev
        dh1raw = (1-h1s[t]*h1s[t])*dh1 # back prop thru tanh        
        dBxh1 += dh1raw  # derivative of Wx+b w.r.t b is 1; d_loss/d_b = d_loss/d_H * d_H/d_b
        dWxh1 += np.dot(dh1raw, xs[t].T) # derivative of Wx+b w.r.t W is x
        dWh1h1 += np.dot(dh1raw,h1s[t-1].T)
        dh1next = np.dot(Wh1h1.T, dh1raw)

    # Can be replaced using LSTM structure
    for dparam in [dWxh1, dWh1h1, dWh1h2, dWh2h2, dWh2h3, dWh3h3, dWh3y, dBxh1, dBh1h2, dBh2h3,dBh3y]:
        np.clip(dparam,-5,5,out=dparam) # mitigate gradient vanish
        
        
    return loss,dWxh1,dWh1h2,dWh1h1,dWh2h3,dWh2h2,dWh3y,dWh3h3,dBxh1,dBh1h2,dBh2h3,dBh3y,h1s[len(inputs)-1],h2s[len(inputs)-1],h3s[len(inputs)-1];


 

In [66]:
# prediction
def sample(h1,h2,h3,seed_ix,n):
    x = np.zeros((vocab_size,1))
    # one hot encode
    x[seed_ix] = 1;
    ixes = [] # empty sentence

    for t in range(n):

        h1 = np.tanh(np.dot(Wxh1 ,x ) + np.dot(Wh1h1,h1) + Bxh1 );
        h2 = np.tanh(np.dot(Wh1h2,h1) + np.dot(Wh2h2,h2) + Bh1h2);
        h3 = np.tanh(np.dot(Wh2h3,h2) + np.dot(Wh3h3,h3) + Bh2h3);
        y = np.dot(Wh3y,h3) + Bh3y;
        p = softmax(y);

        # sample the output
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        # encode this output
        x = np.zeros((vocab_size,1))
        x[ix] = 1;

        ixes.append(ix)
        # if n > 1, it will predict more than 1 subsequent chars
        
    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ("----\n %s \n----" % (txt,))
    
# test
h1prev = np.zeros((hidden_size,1)) # reset RNN memory  
h2prev = np.zeros((hidden_size2,1)) # reset RNN memory  
h3prev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(h1prev,h2prev,h3prev,char_to_ix['a'],200)

----
 CjKEYN.xa-qZwYtYSt~;PlKrXo(TB?-(ds4\?e?B:mY\ Ngupsyr(FMZIhOJK.kt4MKP4Mw,0Em.bJYL"JgmQ(xQ	Y)ofrsV~rGJDir	4-YP;a7o7(5Eh	2cr.1oT JPN)kFPMIRs5a3Y~2ng!
JuQ)u9)Rl,ewksE4w1sp2Z!-Y9Zyd"CFa0)TFwmKdkNrHLkK?1\5d 
----


In [67]:
# Training using Adagrad (decreasing learning rate)
position = 0
inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
print(data[position:position+seq_length])
print("inputs",inputs)
targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
print(data[position+1:position+seq_length+1])
print("targets",targets)



Harry Potter and the Sorc
inputs [73, 8, 66, 66, 38, 28, 67, 21, 11, 11, 49, 66, 28, 8, 1, 46, 28, 11, 30, 49, 28, 23, 21, 66, 25]
arry Potter and the Sorce
targets [8, 66, 66, 38, 28, 67, 21, 11, 11, 49, 66, 28, 8, 1, 46, 28, 11, 30, 49, 28, 23, 21, 66, 25, 49]


In [68]:
# Real training

n, position = 0,0

# for Adaptive Gradient descent
mWxh1 = np.zeros_like(Wxh1);
mWh1h1 = np.zeros_like(Wh1h1);
mWh1h2 = np.zeros_like(Wh1h2);
mWh2h2 = np.zeros_like(Wh2h2);
mWh2h3 = np.zeros_like(Wh2h3);
mWh3h3 = np.zeros_like(Wh3h3);
mWh3y = np.zeros_like(Wh3y);

mBxh1 = np.zeros_like(Bxh1);
mBh1h2 = np.zeros_like(Bh1h2);
mBh2h3 = np.zeros_like(Bh2h3);
mBh3y = np.zeros_like(Bh3y);

smooth_loss = -np.log(1.0/vocab_size)*seq_length;

epoch = 100*1000;
sample_length = 200;

while n<epoch:

    if(position+seq_length+1 >= len(data) or n == 0):
        
        h1prev = np.zeros((hidden_size,1))
        h2prev = np.zeros((hidden_size2,1))
        h3prev = np.zeros((hidden_size,1))
        position = 0;
        
    inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
    
    loss,dWxh1,dWh1h2,dWh1h1,dWh2h3,dWh2h2,dWh3y,dWh3h3,dBxh1,dBh1h2,dBh2h3,dBh3y,h1prev,h2prev,h3prev =\
    lossFunction(inputs,targets,h1prev, h2prev,h3prev)  

    smooth_loss = smooth_loss*0.999+loss*0.001
    
    if(n%1000 == 0):
        print(n,"loss",smooth_loss)
        sample(h1prev,h2prev,h3prev,inputs[0],sample_length);
        
    # update
    for param, dparam, mem in zip([Wxh1,Wh1h1,Wh1h2,Wh2h2,Wh2h3,Wh3h3,Wh3y,Bxh1,Bh1h2,Bh2h3,Bh3y],
                                  [dWxh1,dWh1h1,dWh1h2,dWh2h2,dWh2h3,dWh3h3,dWh3y,dBxh1,dBh1h2,dBh2h3,dBh3y],
                                  [mWxh1,mWh1h1,mWh1h2,mWh2h2,mWh2h3,mWh3h3,mWh3y,mBxh1,mBh1h2,mBh2h3,mBh3y]):
        
        mem += dparam*dparam;
        param += -learning_rate * dparam *  1 / np.sqrt(mem +1e-8) # Adagrad
        
    position += seq_length;
    n += 1;
    
        

0 loss 109.236196265
----
 -UGm'i-eT8P.L04I.U-;xEGLJS"AoFQkbt*U;aGrrtIADHxbr5j5"NBG-CCmS.RSVZ9:-97jhP3\VpPnyPs2-Fdhf49JtCA'z\r,:1eBiLu~u(*P1c?sHzWqf1PQdUpIP;x"679:OGPbQFa~kmlZ93*hDXefM'4ssM\9Wu,P8s
L*(N4o,9mQ)IZI')ZnRe;~5!h6
03 
----
1000 loss 93.3072365399
----
 f   v  dmsasa etsioleilt t ooioemOl rr.tnkee  os wl'ertsa,t ouyosklte
eheoe sn,y slh uf"csaohgd ksue eiaaltn erssbstsi?g ros  bvaaroei uamd s wvrtgier  dieae.yaadt e"gneweu iiw
yy  clgself lrnnida  ic 
----
2000 loss 84.1486283351
----
 ateihende,ate csnhe knd ro slitl.edeo.rcnhrynNedooe nhdire bhnelwe giDntehake. ntera. tbsee augandl .h bpita'e twtiaours

ilh ulck.eosd Bwdgor thg hnteelzobiortodes!A  ib dr hhtH atA iane. l edeeoerr  
----
3000 loss 76.7837769842
----
 ediw lhl oo' tHeeesifae brnl
ifke
"""
WEwt-iheeg aeegtp lok bay poeu,. wafeiwcslt, g".Tdey
  -visbr semsh kerp k hs who grmerwer.ie At. "anrotide bn. 8ons dema.ee dugs likr curtod wherd  4utk,, Oner b 
----
4000 loss 72.8130407837
----
 poiugocke, dan jtcEas ocf

35000 loss 55.7012293707
----
 divbe I soreilas sfeanpsiut!. Jare a nrething was srln anas. Ity as thuse changos ,as salcege sowtt aved in ther bigter.".oerantle (ussoothroit bo mo toieo or nn anen and am, the ne uulfaed axrand the 
----
36000 loss 55.5989015297
----
  Wh thl hapriss hustar theyl hewone mere a bakting the Veod oully Panesad hefs bolg curets?"
Harrahot throng vedts't of abchey,"
"Nook in tAore. errine hadd spot?" s narbe the minn yiseechos you his't 
----
37000 loss 55.1043316111
----
 hey asten of stas Iae laclen. He boswach bagcing wat gayay at the Velring herand vaskey mereed they cunk on, hetcoud to Dumt hamt lo kfeount-ert at the hertin the knst and abarreit and uldoutod" gard. 
----
38000 loss 56.3163380853
----
 po tha cae dad't roocher Pat. I pile't ot nome' ob ine. A betoileg uudsing chYtkent iot choot thees Puniththe teikey ens ontoct yeo his lorssly or har gighunditn fmade somer's jeagh at erounde. tha mi 
----
39000 loss 56.2988414396
----
  a guinbed a rlan

70000 loss 51.9314428757
----
 treand of the bully." SpI very wistle.
"At Madk Harc frster," said Verling at whe Derond and cundor, veak. It limh,"
He sulvirne four,." sadamesmes in throw Must fighto, at ovebe, agofibl gon her a po 
----
71000 loss 51.9107020875
----
 ed. The stalled, them the mafly fin ofe ing, in betinal clakann to hid fabklmusor." sheyghr ckexnind so the cirdtador his meametsly dar, Po clire. Harrytpestnn out and thougvle." Habry dayn be shot of 
----
72000 loss 52.376778663
----
 d"Ha walpsherbed,"t't stateery you bhlsore all dotfee an favsory I srey sromer on yid shanin- magrining had bast tor a sover a throus gat bith bouisury fuener," hered lutle santher in the beecery wead 
----
73000 loss 53.7005915103
----
 d.""
"LI''d him at hey sropes this Hacrionsent uvering toidinge. Ther dweg ose....."
"Sules an sosn, heom, asfer you," a pip bhes staln your Migdt allion.
Whe SfGnand and seal belbpoun though bully fe 
----
74000 loss 53.2616248323
----
 leshad ttwoicuroll

In [74]:
n, position = 0,0

# for Adaptive Gradient descent
mWxh1 = np.zeros_like(Wxh1);
mWh1h1 = np.zeros_like(Wh1h1);
mWh1h2 = np.zeros_like(Wh1h2);
mWh2h2 = np.zeros_like(Wh2h2);
mWh2h3 = np.zeros_like(Wh2h3);
mWh3h3 = np.zeros_like(Wh3h3);
mWh3y = np.zeros_like(Wh3y);

mBxh1 = np.zeros_like(Bxh1);
mBh1h2 = np.zeros_like(Bh1h2);
mBh2h3 = np.zeros_like(Bh2h3);
mBh3y = np.zeros_like(Bh3y);

smooth_loss = -np.log(1.0/vocab_size)*seq_length;

epoch = 100*1000;
sample_length = 200;

while n<epoch:

    if(position+seq_length+1 >= len(data) or n == 0):
        position = 0;
        
    inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position:position+seq_length]] 
    
    loss,dWxh1,dWh1h2,dWh1h1,dWh2h3,dWh2h2,dWh3y,dWh3h3,dBxh1,dBh1h2,dBh2h3,dBh3y,h1prev,h2prev,h3prev =\
    lossFunction(inputs,targets,h1prev, h2prev,h3prev)  

    smooth_loss = smooth_loss*0.999+loss*0.001
    
    if(n%1000 == 0):
        print(n,"loss",smooth_loss)
        sample(h1prev,h2prev,h3prev,inputs[0],sample_length);
        
    # update
    for param, dparam, mem in zip([Wxh1,Wh1h1,Wh1h2,Wh2h2,Wh2h3,Wh3h3,Wh3y,Bxh1,Bh1h2,Bh2h3,Bh3y],
                                  [dWxh1,dWh1h1,dWh1h2,dWh2h2,dWh2h3,dWh3h3,dWh3y,dBxh1,dBh1h2,dBh2h3,dBh3y],
                                  [mWxh1,mWh1h1,mWh1h2,mWh2h2,mWh2h3,mWh3h3,mWh3y,mBxh1,mBh1h2,mBh2h3,mBh3y]):
        
        mem += dparam*dparam;
        param += -learning_rate * dparam *  1 / np.sqrt(mem +1e-8) # Adagrad
        
    position += seq_length;
    n += 1;

0 loss 109.236199473
----
 yhgcjN5 SF'3T oqE!n-w-T"IvSpz-3*5GOn:n.!Df"iA?)x'JEmGtwDf?wZFpIHPJTwKV0rAkgIxTyP);x	j1H-I!u-9oeO!SfehHCW)7E"BXtVw4d!jaO'bV!Uqv!U)TGg~A(MNn9ff
rR(2oHTL"CQTOj'R074iwQE8jio664?AklZjy7hAFjjp~5yeoxz;'~q6rg 
----
1000 loss 59.7784565469
----
 yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyppkkgppwwpuuuuucpkvvvv-pkk--ccup-ucccccuucuc-Gccpwkpppwpppkkkkkpk-kkkpgpkkvcuuuuccgkpppkkkppkp-ccc-cccgppppyyyyyyycckvvoooooooooooooooovcu""cccuuu-ckkkkgpppguppppppp 
----
2000 loss 24.1810057381
----
                                                                                  ttttiiiiiiiiinnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnUGPPx!A!xwwk-gggpppppppppppwwwkkkkkkkkkkkkkkkkkkkgpkkkkk 
----
3000 loss 10.2085633125
----
 eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeecccccccccccccccccccc----kkkkkkkkkkkkkoooooooooooooooooooooooooooooooooooooooooowwooooooooooookkooooooooooooooooooooooooooooooooooooo 
----
4000 loss 4.70933329111
----
 ccccccccccccccccccc-----k

KeyboardInterrupt: 